In [ ]:
import pandas as pd

#all necessary data has to be in a dedicated 'data' folder
data = pd.read_csv('data/data_geo_hos.csv')
#save data for later sanity check
data_check = data
education = pd.read_csv('data/county_data/Education.csv', encoding = "latin")
population = pd.read_csv('data/county_data/PopulationEstimates.csv', encoding = "latin")
unemployment = pd.read_csv('data/county_data/Unemployment.csv', encoding = "latin")
poverty = pd.read_csv('data/county_data/PovertyEstimates.csv', encoding = "latin")

#sort data
data = data.sort_values("county_fips_code")
education = education.sort_values("FIPS Code")
population = population.sort_values("FIPStxt")
unemployment = unemployment.sort_values("FIPS_Code")
poverty = poverty.sort_values("FIPStxt")

#perform combination of data sets (takes ~40 minutes on Intel® Core™ i5-5300U CPU @ 2.30GHz × 4 )
current_fips = data["county_fips_code"].iloc[0]
old_fips = 0

for idx, fips in enumerate(data["county_fips_code"]):

    current_fips = fips
    
    #whenever fips changes, we cache the necessary census data
    if (current_fips != old_fips):
        
        #print current_fips to keep track of progress
        print(current_fips)
        
        pop1 = population.loc[population["FIPStxt"] == fips, "Rural-urban_Continuum Code_2013"]
        pop2 = population.loc[population["FIPStxt"] == fips, "Urban_Influence_Code_2013"]
        pop3 = population.loc[population["FIPStxt"] == fips, "Economic_typology_2015"]
        pop4 = population.loc[population["FIPStxt"] == fips, "POP_ESTIMATE_2019"]
        
        edu1 = education.loc[education["FIPS Code"] == fips, "Percent of adults with less than a high school diploma, 2015-19"]
        edu2 = education.loc[education["FIPS Code"] == fips, "Percent of adults with a high school diploma only, 2015-19"]
        edu3 = education.loc[education["FIPS Code"] == fips, "Percent of adults completing some college or associate's degree, 2015-19"]
        edu4 = education.loc[education["FIPS Code"] == fips, "Percent of adults with a bachelor's degree or higher, 2015-19"]
        
        empl1 = unemployment.loc[(unemployment["FIPS_Code"] == fips) & (unemployment["Attribute"] == "Unemployment_rate_2020"), "Value"]
        empl2 = unemployment.loc[(unemployment["FIPS_Code"] == fips) & (unemployment["Attribute"] == "Median_Household_Income_2019"), "Value"]
        empl3 = unemployment.loc[(unemployment["FIPS_Code"] == fips) & (unemployment["Attribute"] == "Med_HH_Income_Percent_of_State_Total_2019"), "Value"]
       
        pov1 = poverty.loc[(poverty["FIPStxt"] == fips) & (poverty["Attribute"] == "PCTPOVALL_2019"), "Value"]
    
    #census data gets mapped to its respective columns
    data.loc[data.index[idx], "rural-urban_Continuum Code_2013"] = pop1.values[0]
    data.loc[data.index[idx], "urban_influence_code_2013"] = pop2.values[0]
    data.loc[data.index[idx], "economic_typology_2015"] = pop3.values[0]
    data.loc[data.index[idx], "pop_estimate_2019"] = pop4.values[0]
    data.loc[data.index[idx], "less_hs_pct"] = edu1.values[0]
    data.loc[data.index[idx], "hs_only_pct"] = edu2.values[0]
    data.loc[data.index[idx], "some_college_pct"] = edu3.values[0]
    data.loc[data.index[idx], "bachelor_higher_pct"] = edu4.values[0]
    data.loc[data.index[idx], "unempl_rate"] = empl1.values[0]
    data.loc[data.index[idx], "med_hh_income_19"] = empl2.values[0]
    data.loc[data.index[idx], "med_hh_income_pct_state_total_19"] = empl3.values[0]
    data.loc[data.index[idx], "poverty_pct_19"] = pov1.values[0]

    old_fips = current_fips
    
#lastly, perform sanity check

if (len(data_check) == len(data)):
    data.to_csv('data_combined.csv', index=False)
else:
    print("size of data sets is different")